In [1]:
# import libraries
import psycopg2
import pandas as pd

In [2]:
# initialize connection
def initialize_connection(host, db_name, user, password):

    try:
        conn_str = f"host={host} dbname={db_name} user={user} password={password}"
        conn = psycopg2.connect(conn_str)
        conn.set_session(autocommit=True)
        cur = conn.cursor()
        print(f"Connection Initialized with {db_name}")
        return conn, cur
    except psycopg2.errors as error:
        print(f"Unable to initialize connection: {error}")
        exit()

In [3]:
def create_database(conn, cur, db_name):

    try:
        drop_db_str = f"DROP DATABASE IF EXISTS {db_name}"
        create_db_str = f"CREATE DATABASE {db_name}"
        cur.execute(drop_db_str)
        cur.execute(create_db_str)
        return f"{db_name} Database Created"
    except psycopg2.errors as error:
        print(f"Unable to create database: {error}")
        exit()

In [4]:
def create_table(conn, cur, table_name, table_query):

    try:
        cur.execute(table_query)
        print(f"{table_name} table created.")
    except psycopg2.errors as error:
        print(f"Unable to create table: {error}")
        exit()

In [5]:
def prepare_data(file_path, required_columns):
    df = pd.read_csv(file_path)
    if len(required_columns) == 0:
        return df
    else:
        df_clean = df[required_columns]
        return df_clean

In [6]:
def insert_data(conn, cur, df, table_name, table_query):

    try:
        for i, row in df.iterrows():
            cur.execute(table_query, list(row))
        print(f"Data inserted for {table_name}.")
    except Exception as error:
        print(f"Unable to insert rows for {table_name}: {error}")
        exit()

In [7]:
# query for creating country database

create_country = "CREATE TABLE IF NOT EXISTS country(country_code VARCHAR PRIMARY KEY,\
                  short_name VARCHAR, table_name VARCHAR, long_name VARCHAR, currency_unit VARCHAR)"

# query for creating account_data database

create_account_data = "CREATE TABLE IF NOT EXISTS account_data(country_name VARCHAR,\
                       country_code VARCHAR, series_name VARCHAR, series_code VARCHAR,\
                       year_2016 numeric, year_2017 numeric, year_2018 numeric)"

# query for creating account_series database

create_account_series = "CREATE TABLE IF NOT EXISTS account_series(code VARCHAR, topic VARCHAR, indicator_name VARCHAR)"

tables = {"country": create_country, "account_data": create_account_data, "account_series": create_account_series}

In [8]:
# declaring files and columns for loading
country_file_path = "/dummy/data/Wealth-AccountsCountry.csv"
country_columns = ['Code', 'Short Name', 'Table Name', 'Long Name', 'Currency Unit']

account_data_file_path = "/dummy/data/Wealth-AccountData.csv"
account_data_columns = ['Country Name', 'Country Code', 'Series Name', 'Series Code', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]']

account_series_file_path = "/dummy/data/Wealth-AccountSeries.csv"
account_series_columns = ['Code', 'Topic', 'Indicator Name']

country_df = prepare_data(country_file_path, country_columns)
account_data_df = prepare_data(account_data_file_path, account_data_columns)
account_series_df = prepare_data(account_series_file_path, account_series_columns)

In [9]:
# query for inserting data into tables

country_data = "INSERT INTO country(country_code, short_name, table_name, long_name, currency_unit)\
                VALUES(%s, %s, %s, %s, %s)"

account_data = "INSERT INTO account_data(country_name, country_code, series_name, series_code, year_2016, year_2017, year_2018)\
                VALUES(%s, %s, %s, %s, %s, %s, %s)"

account_series = "INSERT INTO account_series(code, topic, indicator_name)\
                VALUES(%s, %s, %s)"

data = {"country": [country_data, country_df], "account_data": [account_data, account_data_df], "account_series": [account_series, account_series_df]}

In [10]:
def main():
    conn, cur =  initialize_connection(host='127.0.0.1', db_name='postgres', user='dummy', password='dummy')
    print(create_database(conn, cur, db_name="accounts"))
    cur.close()
    conn.close()

    conn, cur = initialize_connection(host='127.0.0.1', db_name='accounts', user='dummy', password='dummy')
    for k,v in tables.items():
        create_table(conn, cur, k, v)

    for k,v in data.items():
        insert_data(conn, cur, v[1], k, v[0])
    
    cur.close()
    conn.close()

In [11]:
main()

Connection Initialized with postgres
accounts Database Created
Connection Initialized with accounts
country table created.
account_data table created.
account_series table created.
Data inserted for country.
Data inserted for account_data.
Data inserted for account_series.
